In [ ]:
import pandas as pd
import os
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
import re
from tensorflow.keras.applications.resnet50 import preprocess_input

In [ ]:
df = pd.read_csv("transformed_radiological_gradings_updated.csv")

In [ ]:
image_folder = "all_images"

In [ ]:
image_list = []
label_list = []

for image_filename in os.listdir(image_folder):
    if image_filename.endswith(".png"):
        patient_number = re.search(r'^(\d+)_', image_filename).group(1)
        patient_row = df[df['Patient'] == int(patient_number)]
        image = Image.open(os.path.join(image_folder, image_filename))

        # Resize image to 224x224 pixels
        image = image.resize((224,224))

        # Convert image to RGB
        image = image.convert("RGB")

        # Convert the image to numpy array
        image_array = np.array(image)

        # Normalize array
        image_array = preprocess_input(image_array)

        image_list.append(image_array)
        labels = patient_row.iloc[0].values[1:]
        label_list.append(labels)
        
        #print(f"Image: {image_filename}, Patient Data: {labels}")

image_list_np = np.array(image_list)

In [ ]:
modic_labels = [row[0] for row in label_list]
up_endplate_labels = [row[1] for row in label_list]
low_endplate_labels = [row[2] for row in label_list]
spondylolisthesis_labels = [row[3] for row in label_list]
herniation_labels = [row[4] for row in label_list]
narrowing_labels = [row[5] for row in label_list]
bulging_labels = [row[6] for row in label_list]
pfirrman_labels = [row[7] for row in label_list]

In [ ]:
label_dict = {
    'disc_herniation': np.array(herniation_labels),
    'disc_narrowing': np.array(narrowing_labels),
    'disc_bulging': np.array(bulging_labels),
    'pfirrman_grade': np.array(pfirrman_labels),
    'up_endplate': np.array(up_endplate_labels),
    'low_endplate': np.array(low_endplate_labels),
    'spondylolisthesis': np.array(spondylolisthesis_labels),
    #'modic_type': np.array(modic_labels_one_hot)
}

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable=False

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)

In [ ]:
herniation_output = Dense(1, activation='sigmoid', name='disc_herniation')(x)
narrowing_output = Dense(1, activation='sigmoid', name='disc_narrowing')(x)
bulging_output = Dense(1, activation='sigmoid', name='disc_bulging')(x)
#modic_output = Dense(4, activation='softmax', name='modic_type')(x)
pfirrman_output = Dense(1, activation='linear', name='pfirrman_grade')(x)
up_endplate_output = Dense(1, activation='sigmoid', name='up_endplate')(x)
low_endplate_output = Dense(1, activation='sigmoid', name='low_endplate')(x)
spondylolisthesis_output = Dense(1, activation='sigmoid', name='spondylolisthesis')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=[
    herniation_output,
    narrowing_output,
    bulging_output,
    #modic_output,
    pfirrman_output,
    up_endplate_output,
    low_endplate_output,
    spondylolisthesis_output
])

In [ ]:
optimizer = Adam(learning_rate=0.0001)
model.compile(
    optimizer=optimizer,
    loss={
        'disc_herniation': 'binary_crossentropy',
        'disc_narrowing': 'binary_crossentropy',
        'disc_bulging': 'binary_crossentropy',
        #'modic_type': 'sparse_categorical_crossentropy',
        'pfirrman_grade': 'mean_squared_error',
        'up_endplate': 'binary_crossentropy',
        'low_endplate': 'binary_crossentropy',
        'spondylolisthesis': 'binary_crossentropy'
    },
    metrics={
        'disc_herniation': 'accuracy',
        'disc_narrowing': 'accuracy',
        'disc_bulging': 'accuracy',
        #'modic_type': 'accuracy',
        'pfirrman_grade': 'mae',
        'up_endplate': 'accuracy',
        'low_endplate': 'accuracy',
        'spondylolisthesis': 'accuracy'
    }
)

In [ ]:
#modic_labels_one_hot = to_categorical(modic_labels, num_classes=4)

In [ ]:
print("First Image Shape:", image_list[0].shape)
#print("Modic Labels (One-Hot):", modic_labels_one_hot[0])
print("UP Endplate Label:", up_endplate_labels[0])
print("LOW Endplate Label:", low_endplate_labels[0])
print("Spondylolisthesis Label:", spondylolisthesis_labels[0])
print("Disc Herniation Label:", herniation_labels[0])
print("Disc Narrowing Label:", narrowing_labels[0])
print("Disc Bulging Label:", bulging_labels[0])
print("Pfirmann Label:", pfirrman_labels[0])

In [ ]:
label_dict = {
    'disc_herniation': np.array(herniation_labels),
    'disc_narrowing': np.array(narrowing_labels),
    'disc_bulging': np.array(bulging_labels),
    'pfirrman_grade': np.array(pfirrman_labels),
    'up_endplate': np.array(up_endplate_labels),
    'low_endplate': np.array(low_endplate_labels),
    'spondylolisthesis': np.array(spondylolisthesis_labels),
    #'modic_type': np.array(modic_labels_one_hot)
}

In [ ]:
history = model.fit(
    image_list_np, label_dict,
    batch_size=32,
    epochs=100,
    verbose=1
)

In [ ]:
print(image_list_np.shape)
print(np.array(modic_labels_one_hot).shape)

In [ ]:
for key, value in label_dict.items():
    print(f"{key} Shape:", np.array(value).shape)

In [ ]:
image = Image.open(os.path.join(image_folder, '100_t1.png'))

# Resize image to 224x224 pixels
image_resized = image.resize((224,224))

# Convert image to RGB
image_recolored = image_resized.convert("RGB")

# Convert the image to numpy array
image_array = np.array(image_recolored)

# Normalize array
image_array = preprocess_input(image_array)
image_array = np.expand_dims(image_array, 0)
predictions = model.predict(image_array)

In [ ]:
print("Disc Herniation: ")
print(predictions[0][0][0])
print("Disc Narrowing: ")
print(predictions[1][0][0])
print("Disc Bulging: ")
print(predictions[2][0][0])
print("Pfirmann Grade: ")
print(predictions[3][0][0])
print("Up-Endplate: ")
print(predictions[4][0][0])
print("Low-Endplate: ")
print(predictions[5][0][0])
print("Spondylo: ")
print(predictions[6][0][0])


In [ ]:
model.summary()